In [ ]:
import pandas as pd
import json
import glob

In [ ]:
datapath = './Data'

In [ ]:
files = glob.glob(datapath+'/*chain*.json')
files.sort()

In [ ]:
def extract_openinterest(data_in):
    option_type = ['callExpDateMap', 'putExpDateMap']
    data_out = pd.DataFrame(columns=['optiontype', 'date', 'strike', 'openinterest'])
    for option in option_type:
        dates = data_in[option].keys()
        for date in dates:
            openinterest = []
            strikes = data_in[option][date].keys()
            for strike in strikes:
                openinterest.append(data_in[option][date][strike][0]['openInterest'])
            temp = pd.DataFrame()
            temp['openinterest'] = openinterest
            temp['strike'] = strikes
            temp['date'] = date.split(':')[0]
            temp['optiontype'] = option[:-10]
            data_out = data_out.append(temp)
    return data_out

In [ ]:
data_in = pd.read_json(files[0])
data_out = extract_openinterest(data_in)

In [ ]:
d1.to_excel('./Data/processed_chain.xlsx', index=False)

In [ ]:
data_out[data_out['optiontype']=='call'].sort_values('openinterest', ascending=False)

In [ ]:
data_out[data_out['optiontype']=='put'].sort_values('openinterest', ascending=False)

In [ ]:
data_out.sort_values('openinterest', ascending=False).head(10)

In [ ]:
data_in = pd.read_json(files[1])
d2 = extract_openinterest(data_in)

In [ ]:
d2.sort_values('openinterest', ascending=False).head(10)

In [ ]:
data_out['name'] = data_out.apply(lambda x: (x.optiontype+'_'+x.date+'_'+x.strike), axis=1)
d2['name'] = d2.apply(lambda x: (x.optiontype+'_'+x.date+'_'+x.strike), axis=1)

In [ ]:
data_out = data_out[['name', 'openinterest']]

In [ ]:
d2 = d2[['name', 'openinterest']]

In [ ]:
merged_df = pd.merge(data_out, d2, on='name', suffixes=("_before", "_after"))

In [ ]:
merged_df['change'] = merged_df.apply(lambda x: x.openinterest_after-x.openinterest_before, axis=1)

In [ ]:
merged_df.sort_values('change', ascending=False)

In [ ]:
merged_df.sort_values('change', ascending=True)